In [1]:
import pandas as pd

cols = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
        'barely_true_counts', 'false_counts', 'half_true_counts',
        'mostly_true_counts', 'pants_on_fire_counts', 'context']

train_df = pd.read_csv("sample_data/train.tsv", sep='\t', names=cols)
test_df = pd.read_csv("sample_data/test.tsv", sep='\t', names=cols)
val_df = pd.read_csv("sample_data/valid.tsv", sep='\t', names=cols)

In [2]:
def binary_label(label):
    fake_labels = ['false', 'barely-true', 'pants-fire']
    return 1 if label in fake_labels else 0

train_df['label'] = train_df['label'].apply(binary_label)
val_df['label'] = val_df['label'].apply(binary_label)
test_df['label'] = test_df['label'].apply(binary_label)

In [3]:
!pip install transformers datasets
!pip install --upgrade transformers
!pip install numpy==1.26

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['statement'], padding='max_length', truncation=True, max_length=128)

from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df[['statement', 'label']])
val_dataset = Dataset.from_pandas(val_df[['statement', 'label']])
test_dataset = Dataset.from_pandas(test_df[['statement', 'label']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/10240 [00:00<?, ? examples/s]

Map:   0%|          | 0/1284 [00:00<?, ? examples/s]

Map:   0%|          | 0/1267 [00:00<?, ? examples/s]

In [5]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# training_args = TrainingArguments(
#     output_dir="./bert_fake_news",
#     evaluation_strategy="epoch",
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     save_strategy="epoch",
#     logging_dir="./logs",
# )
training_args = TrainingArguments(
    output_dir="./bert_fake_news",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: maheenrizwan10 (maheenrizwan10-iba-institute-of-business-administration) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.672600
1000,0.599200
1500,0.462800


TrainOutput(global_step=1920, training_loss=0.5186784982681274, metrics={'train_runtime': 754.5595, 'train_samples_per_second': 40.712, 'train_steps_per_second': 2.545, 'total_flos': 2020692905164800.0, 'train_loss': 0.5186784982681274, 'epoch': 3.0})

In [8]:
model.save_pretrained("bert_fake_news_model")
tokenizer.save_pretrained("bert_fake_news_model")

('bert_fake_news_model/tokenizer_config.json',
 'bert_fake_news_model/special_tokens_map.json',
 'bert_fake_news_model/vocab.txt',
 'bert_fake_news_model/added_tokens.json')

In [11]:
!zip -r bert_fake_news_model.zip bert_fake_news_model
from google.colab import files
files.download('bert_fake_news_model.zip')

  adding: bert_fake_news_model/ (stored 0%)
  adding: bert_fake_news_model/special_tokens_map.json (deflated 42%)
  adding: bert_fake_news_model/vocab.txt (deflated 53%)
  adding: bert_fake_news_model/model.safetensors (deflated 7%)
  adding: bert_fake_news_model/tokenizer_config.json (deflated 75%)
  adding: bert_fake_news_model/config.json (deflated 49%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>